In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum
from pyspark.sql import functions as F
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
# Inisialisasi SparkSession
spark = SparkSession.builder \
    .appName("Membuat DataFrame dari CSV") \
    .config("spark.storage.memoryFraction", "0.4") \
    .config("spark.memory.fraction", "0.6") \
    .getOrCreate()
    



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/10 11:15:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#DARAFRAME DARI CSV0.

df = spark.read.csv("train.csv", header=True, inferSchema=True)

24/11/10 11:15:31 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
# print shema dataframe
df.printSchema()

root
 |-- key: timestamp (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- passenger_count: integer (nullable = true)



In [5]:
#print data frame
df.show(5)

+-------------------+-----------+-------------------+----------------+---------------+-----------------+----------------+---------------+
|                key|fare_amount|    pickup_datetime|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|passenger_count|
+-------------------+-----------+-------------------+----------------+---------------+-----------------+----------------+---------------+
|2009-06-15 17:26:21|        4.5|2009-06-15 17:26:21|      -73.844311|      40.721319|        -73.84161|       40.712278|              1|
|2010-01-05 16:52:16|       16.9|2010-01-05 16:52:16|      -74.016048|      40.711303|       -73.979268|       40.782004|              1|
|2011-08-18 00:35:00|        5.7|2011-08-18 00:35:00|      -73.982738|       40.76127|       -73.991242|       40.750562|              2|
|2012-04-21 04:30:42|        7.7|2012-04-21 04:30:42|       -73.98713|      40.733143|       -73.991567|       40.758092|              1|
|2010-03-09 07:51:00|        5.3|2

In [6]:
# cek nilai kosong
null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.show()

+---+-----------+---------------+----------------+---------------+-----------------+----------------+---------------+
|key|fare_amount|pickup_datetime|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|passenger_count|
+---+-----------+---------------+----------------+---------------+-----------------+----------------+---------------+
|  0|          0|              0|               0|              0|              376|             376|              0|
+---+-----------+---------------+----------------+---------------+-----------------+----------------+---------------+



In [7]:
kolom_null = [
    'dropoff_longitude',
    'dropoff_latitude',
] 
dictio = {}
for kol in kolom_null:
    modus = df.groupBy(kol).count().orderBy(F.desc('count')).first()[0]
    dictio[kol] = modus

In [8]:
#isi data kosong
df = df.fillna(dictio)

In [9]:
# cek nilai kosong
null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.show()

+---+-----------+---------------+----------------+---------------+-----------------+----------------+---------------+
|key|fare_amount|pickup_datetime|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|passenger_count|
+---+-----------+---------------+----------------+---------------+-----------------+----------------+---------------+
|  0|          0|              0|               0|              0|                0|               0|              0|
+---+-----------+---------------+----------------+---------------+-----------------+----------------+---------------+



In [10]:
#drop kolom ket dan pickup date
df = df.drop('key', 'pickup_datetime')

In [11]:
df.show(5)

+-----------+----------------+---------------+-----------------+----------------+---------------+
|fare_amount|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|passenger_count|
+-----------+----------------+---------------+-----------------+----------------+---------------+
|        4.5|      -73.844311|      40.721319|        -73.84161|       40.712278|              1|
|       16.9|      -74.016048|      40.711303|       -73.979268|       40.782004|              1|
|        5.7|      -73.982738|       40.76127|       -73.991242|       40.750562|              2|
|        7.7|       -73.98713|      40.733143|       -73.991567|       40.758092|              1|
|        5.3|      -73.968095|      40.768008|       -73.956655|       40.783762|              1|
+-----------+----------------+---------------+-----------------+----------------+---------------+
only showing top 5 rows



In [12]:
#konversi ke radians longitude dan latitude

df = df.withColumn("pickup_longitude", F.radians(col("pickup_longitude")))
df = df.withColumn("pickup_latitude", F.radians(col("pickup_latitude")))
df = df.withColumn("dropoff_longitude", F.radians(col("dropoff_longitude")))
df = df.withColumn("dropoff_latitude", F.radians(col("dropoff_latitude")))

In [13]:
df.show(5)

+-----------+-------------------+------------------+-------------------+------------------+---------------+
|fare_amount|   pickup_longitude|   pickup_latitude|  dropoff_longitude|  dropoff_latitude|passenger_count|
+-----------+-------------------+------------------+-------------------+------------------+---------------+
|        4.5|-1.2888263608166666|0.7107210923049249|  -1.28877921947357| 0.710563297087252|              1|
|       16.9|-1.2918237369141639| 0.710546280127045|-1.2911818048152806|0.7117802453614975|              1|
|        5.7|-1.2912423677403246|0.7114183687943891|-1.2913907905399142|0.7112314789380856|              2|
|        7.7| -1.291319022601072|0.7109274600356805|-1.2913964628599832|0.7113629022307606|              1|
|        5.3|-1.2909867991779551|0.7115359690793884|-1.2907871335115269|0.7118109282497477|              1|
+-----------+-------------------+------------------+-------------------+------------------+---------------+
only showing top 5 rows



In [14]:
def jarak(pikcup_lat, pickup_long, dropoff_lat, dropoff_long):
    R = 6371
    lat1 = pikcup_lat
    lon1 = pickup_long
    lat2 = dropoff_lat
    lon2 = dropoff_long
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = F.sin(dlat/2)**2 + F.cos(lat1) * F.cos(lat2) * F.sin(dlon/2)**2
    c = 2 * F.asin(F.sqrt(a))
    return R * c

In [15]:
#buat kolom jarak
df = df.withColumn("jarak", jarak(df.pickup_latitude, df.pickup_longitude, df.dropoff_latitude, df.dropoff_longitude))

In [16]:
df.show(5)

+-----------+-------------------+------------------+-------------------+------------------+---------------+------------------+
|fare_amount|   pickup_longitude|   pickup_latitude|  dropoff_longitude|  dropoff_latitude|passenger_count|             jarak|
+-----------+-------------------+------------------+-------------------+------------------+---------------+------------------+
|        4.5|-1.2888263608166666|0.7107210923049249|  -1.28877921947357| 0.710563297087252|              1| 1.030763935049508|
|       16.9|-1.2918237369141639| 0.710546280127045|-1.2911818048152806|0.7117802453614975|              1| 8.450133595805992|
|        5.7|-1.2912423677403246|0.7114183687943891|-1.2913907905399142|0.7112314789380856|              2|1.3895252257697308|
|        7.7| -1.291319022601072|0.7109274600356805|-1.2913964628599832|0.7113629022307606|              1| 2.799270239983641|
|        5.3|-1.2909867991779551|0.7115359690793884|-1.2907871335115269|0.7118109282497477|              1|1.99

In [17]:
# drop kolom pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude
df = df.drop('pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude')

In [18]:
df.show(5)

+-----------+---------------+------------------+
|fare_amount|passenger_count|             jarak|
+-----------+---------------+------------------+
|        4.5|              1| 1.030763935049508|
|       16.9|              1| 8.450133595805992|
|        5.7|              2|1.3895252257697308|
|        7.7|              1| 2.799270239983641|
|        5.3|              1|1.9991567879963377|
+-----------+---------------+------------------+
only showing top 5 rows



In [19]:
#hitung korelasi
correlation_fare_amount_jarak = df.stat.corr("fare_amount", "jarak")
correlation_fare_amount_passenger_count = df.stat.corr("fare_amount", "passenger_count")

In [20]:
print(f"korelasi fare_amount_jarak {correlation_fare_amount_jarak}")
print(f"korelaso fare amount dan passenger_count {correlation_fare_amount_passenger_count}")

korelasi fare_amount_jarak 0.012939121700926964
korelaso fare amount dan passenger_count 0.006131483315520129


In [21]:
#Machine learningh SEssion
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [22]:
spark = SparkSession.builder \
    .appName("model ML") \
    .config("spark.storage.memoryFraction", "0.4") \
    .config("spark.memory.fraction", "0.6") \
    .getOrCreate()

24/11/10 11:25:56 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [23]:
#rubah data ke bentuk vektor
vf = VectorAssembler(inputCols=["jarak","passenger_count"], outputCol="features")
df_vf = vf.transform(df)

In [24]:
df_vf.select("features").show(truncate=False)

+-------------------------+
|features                 |
+-------------------------+
|[1.030763935049508,1.0]  |
|[8.450133595805992,1.0]  |
|[1.3895252257697308,2.0] |
|[2.799270239983641,1.0]  |
|[1.9991567879963377,1.0] |
|[3.7872390847819912,1.0] |
|[1.5558066593500526,1.0] |
|[4.155444291845812,1.0]  |
|[1.253231512725298,1.0]  |
|[2.8496267961070503,2.0] |
|[1.374576999593841,1.0]  |
|[0.0,3.0]                |
|[0.7177674874531168,2.0] |
|[2.0193838271979137,1.0] |
|[1.6714445980753232,2.0] |
|[0.0,1.0]                |
|[2.609774493745362,1.0]  |
|[1.5119042477189055,1.0] |
|[0.9219517182545394,1.0] |
|[0.39917857039885574,1.0]|
+-------------------------+
only showing top 20 rows



In [25]:
#normalisasi 
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
scale_df = scaler.fit(df_vf).transform(df_vf)


In [26]:
train_data, test_data = scale_df.randomSplit([0.8, 0.2], seed=12345)

In [27]:
train_data.select("features","scaled_features").show(truncate=False)

+-------------------------+--------------------------------------------+
|features                 |scaled_features                             |
+-------------------------+--------------------------------------------+
|[0.03048591960311607,1.0]|[1.54838886738623E-6,0.004807692307692308]  |
|[0.0725502844614506,1.0] |[3.6848503915339342E-6,0.004807692307692308]|
|[16.447528746811816,1.0] |[8.353748464578158E-4,0.004807692307692308] |
|[9.695039003207652,1.0]  |[4.924138965421802E-4,0.004807692307692308] |
|[0.0,5.0]                |[0.0,0.02403846153846154]                   |
|[6.983482474435493,3.0]  |[3.5469313898923717E-4,0.014423076923076924]|
|[1.2142596363126945,1.0] |[6.167260582786724E-5,0.004807692307692308] |
|[2.430373687660271,1.0]  |[1.2343939794346684E-4,0.004807692307692308]|
|[3.2470794209034395,1.0] |[1.649201235291557E-4,0.004807692307692308] |
|[1.4699590445869959,3.0] |[7.46596543513668E-5,0.014423076923076924]  |
|[0.8058296471758282,4.0] |[4.0928325959676485E-5,0

In [28]:
lr = LinearRegression(featuresCol="scaled_features", labelCol="fare_amount")

In [29]:
#train model
model = lr.fit(train_data)

24/11/10 11:27:47 WARN Instrumentation: [5f75ab9a] regParam is zero, which might cause numerical instability and overfitting.
24/11/10 11:28:05 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/11/10 11:28:05 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/11/10 11:31:00 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [30]:
#evaluasi training
predictions_train = model.transform(train_data)
evaluator = RegressionEvaluator(labelCol="fare_amount", predictionCol="prediction", metricName="rmse")
rmse_train = evaluator.evaluate(predictions_train)
print(f"RMSE Train: {rmse_train}")

RMSE Train: 19.516112799571808


In [ ]:
#evaluasi test
predictions_test = model.transform(test_data)
rmse_test = evaluator.evaluate(predictions_test)
print(f"RMSE Test: {rmse_test}")

RMSE Test: 24.911890060064117


In [34]:
#SECTION OPTIMASI
import math
import numpy as np
import multiprocessing as mp
import os
mp.freeze_support()

os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

In [ ]:
#optimasi menggunakan KMA

search_space = {
    "regparam":[0,1],
    "elasticnetparam":[0,1],
    "solver":[0,1]
}


def fitness_function(individu):
    regparam_dict = {
        1:0,
        2:0.01,
        3:0.1,
        4:0.5,
        5:1
    }
    
    elasticnetparam_dict = {
        1:0,
        2:0.01,
        3:0.1,
        4:0.5,
        5:1
    }
    
    solver_dict = {
        1:"auto",
        2:"normal",
        3:"l-bfgs"
    }
    
    lower_bond = np.array([1,1,1])
    upper_bond = np.array([5,5,3])
    delta = (upper_bond - lower_bond)
    term = individu * delta
    hasil = np.int64(np.floor(lower_bond + term))
    
    regparam = regparam_dict[hasil[0]]
    elasticnetparam = elasticnetparam_dict[hasil[1]]
    solver = solver_dict[hasil[2]]
    
    lr = LinearRegression(featuresCol="scaled_features", labelCol="fare_amount", regParam=regparam, elasticNetParam=elasticnetparam, solver=solver)
    model = lr.fit(train_data)
    predictions_train = model.transform(train_data)
    predictions_test = model.transform(test_data)
    
    evaluator = RegressionEvaluator(labelCol="fare_amount", predictionCol="prediction", metricName="rmse")
    rmse_train = evaluator.evaluate(predictions_train)
    rmse_test = evaluator.evaluate(predictions_test)
    
    delta_rmse = abs(rmse_train - rmse_test)
    fit_value = (1/(1+rmse_test)) * (1/(1+delta_rmse))
    
    return fit_value

class KomodoMlipirAlgorithm:
    """
    Implementasi Algoritma Komodo Mlipir.

    Metode oleh:
        Prof. Dr. Suyanto, S.T., M.Sc. (2021)

    Implementasi Python oleh:
        Pejalan Sunyi (2024)

    Parameters:
        n (int): Jumlah populasi.
        p (float): Proporsi jantan besar.
        d (float): Tingkat mlipir.
        fitness_function (callable): Fungsi fitness yang akan dioptimalkan.
        search_space (dict): Batasan pencarian untuk setiap parameter.
        max_iter (int): Jumlah iterasi maksimum.
        random_state (int): Seed untuk keacakan.
        problem (str): Jenis masalah, "maximize" atau "minimize".
        alpha (float): Radius parthenogenesis.
        stop_criteria (float): Kriteria penghentian.
    """

    def __init__(
        self,
        n: int,
        p: float,
        d: float,
        fitness_function: callable,
        search_space: dict,
        max_iter: int = 100,
        random_state: int = 42,
        problem: str = "maximize",
        alpha: float = 0.1,
        stop_criteria: float = None,
    ):
        self.n = n
        self.p = p
        self.d = d
        self.fitness_function = fitness_function
        self.search_space = search_space
        self.max_iter = max_iter
        self.random_state = random_state
        self.problem = problem.lower()
        self.alpha = alpha
        self.stop_criteria = stop_criteria

        self.population = []
        self.male_large = []
        self.male_small = []
        self.female = None

        self.male_large_fitness = []
        self.male_small_fitness = []
        self.female_fitness = None

        self.history = {"best_fitness": [], "best_solution": []}
        self.best_fitness = None
        self.best_solution = None

        self.rng = np.random.default_rng(self.random_state)

    def initialize_population(self):
        """
        Inisialisasi populasi awal secara acak berdasarkan search_space.
        """
        for _ in range(self.n):
            individual = np.array([
                self.rng.uniform(low, high)
                for low, high in self.search_space.values()
            ])
            self.population.append(individual)
            
    def calculate_fitness(self):
        """
        Menghitung nilai fitness untuk setiap individu dalam populasi
       
        """
        
        fitness_values = np.array([
            self.fitness_function(individual)
            for individual in self.population
        ])
        return fitness_values
    
    def sort_population(self, fitness_values):
        """
        Mengurutkan populasi berdasarkan nilai fitness.

        Args:
            fitness_values (np.array): Array nilai fitness.

        Returns:
            Tuple: Populasi dan fitness yang sudah diurutkan.
        """
        if self.problem == "maximize":
            sorted_indices = np.argsort(fitness_values)[::-1]
        elif self.problem == "minimize":
            sorted_indices = np.argsort(fitness_values)
        else:
            raise ValueError("Parameter 'problem' harus 'maximize' atau 'minimize'.")

        sorted_population = [self.population[i] for i in sorted_indices]
        sorted_fitness = fitness_values[sorted_indices]
        return sorted_population, sorted_fitness

    def split_population(self, sorted_population, sorted_fitness):
        """
        Membagi populasi menjadi jantan besar, betina, dan jantan kecil.

        Args:
            sorted_population (list): Populasi yang sudah diurutkan.
            sorted_fitness (np.array): Fitness yang sudah diurutkan.
        """
        num_male_large = math.floor(self.n * self.p)
        self.male_large = sorted_population[:num_male_large]
        self.male_large_fitness = sorted_fitness[:num_male_large]

        self.female = sorted_population[num_male_large]
        self.female_fitness = sorted_fitness[num_male_large]

        self.male_small = sorted_population[num_male_large + 1:]
        self.male_small_fitness = sorted_fitness[num_male_large + 1:]

    def male_large_movement(self):
        """
        Mengupdate posisi jantan besar berdasarkan interaksi antar jantan besar.

        Returns:
            list: Daftar individu jantan besar yang telah diperbarui.
        """
        updated_males = []
        for i, male in enumerate(self.male_large):
            movement = np.zeros_like(male)
            for j, other_male in enumerate(self.male_large):
                if i != j:
                    r1 = self.rng.normal(0.5, 0.1)
                    r2 = self.rng.normal(0.5, 0.1)
                    if self.problem == "maximize":
                        if (
                            self.male_large_fitness[j] > self.male_large_fitness[i]
                            and r2 < 0.5
                        ):
                            movement += r1 * (other_male - male)
                        else:
                            movement += r1 * (male - other_male)
                    else:
                        if (
                            self.male_large_fitness[j] < self.male_large_fitness[i]
                            and r2 < 0.5
                        ):
                            movement += r1 * (other_male - male)
                        else:
                            movement += r1 * (male - other_male)
                            
            updated_male = male + movement
            updated_male = self.clip_individual(updated_male)
            updated_males.append(updated_male)
        return updated_males

    def female_movement(self):
        """
        Mengupdate posisi betina melalui kawin atau parthenogenesis.

        Returns:
            np.array: Individu betina yang telah diperbarui.
        """
        r = self.rng.uniform()
        r1 = self.rng.normal(0.5, 0.1)
        if r < 0.5:
            # Kawin dengan jantan besar terbaik
            male = self.male_large[0]
            offspring1 = r1 * self.female + (1 - r1) * male
            offspring2 = r1 * male + (1 - r1) * self.female

            fitness1 = self.fitness_function(offspring1)
            fitness2 = self.fitness_function(offspring2)
            
            if self.problem == "maximize":
                offspring = offspring1 if fitness1 > fitness2 else offspring2
            else :
                offspring = offspring1 if fitness1 < fitness2 else offspring2
                
            print("Betina kawin dengan jantan besar.")
        else:
            # Parthenogenesis
            ub_lb = np.array([
                high - low
                for low, high in self.search_space.values()
            ])
            offspring = self.female + ub_lb * ((2 * r1 - 1) * self.alpha)
            offspring = self.clip_individual(offspring)
            print("Betina melakukan parthenogenesis.")
        return offspring

    def male_small_movement(self):
        """
        Mengupdate posisi jantan kecil berdasarkan interaksi dengan jantan besar.

        Returns:
            list: Daftar individu jantan kecil yang telah diperbarui.
        """
        updated_males = []
        for male in self.male_small:
            movement = np.zeros_like(male)
            for large_male in self.male_large:
                r1 = self.rng.normal(0.5, 0.1)
                r2 = self.rng.uniform()
                if r2 < self.d:
                    movement += r1 * (large_male - male)
            updated_male = male + movement
            updated_male = self.clip_individual(updated_male)
            updated_males.append(updated_male)
        return updated_males

    def clip_individual(self, individual):
        """
        Membatasi nilai individu agar tetap dalam batas search_space.

        Args:
            individual (np.array): Individu yang akan dibatasi.

        Returns:
            np.array: Individu yang telah dibatasi.
        """
        clipped = np.array([
            np.clip(value, low, high)
            for value, (low, high) in zip(individual, self.search_space.values())
        ])
        return clipped

    def optimize(self):
        """
        Menjalankan proses optimasi menggunakan Algoritma Komodo Mlipir.

        Returns:
            dict: Solusi terbaik dan nilai fitness terbaik.
        """
        self.initialize_population()
        for iteration in range(self.max_iter):
            if iteration == 0:
                fitness_values = self.calculate_fitness()
                sorted_pop, sorted_fit = self.sort_population(fitness_values)
            self.split_population(sorted_pop, sorted_fit)

            # Pergerakan masing-masing kelompok
            new_male_large = self.male_large_movement()
            new_female = self.female_movement()
            new_male_small = self.male_small_movement()

            # Membentuk populasi baru
            old_population = self.population.copy()
            old_fitness_values = fitness_values.copy()
            self.population = new_male_large + [new_female] + new_male_small
            child_fitness_values = self.calculate_fitness()
            self.population = old_population + self.population
            fitness_values_new = np.concatenate([old_fitness_values, child_fitness_values])
            new_populasi, fitness_values = self.sort_population(fitness_values_new)
            self.population = new_populasi[:self.n]
            fitness_values = fitness_values[:self.n]
            sorted_pop = self.population
            sorted_fit = fitness_values
            

            # Evaluasi populasi baru
            self.best_fitness = fitness_values[0]
            self.best_solution = self.population[0]

            # Menyimpan sejarah
            self.history["best_fitness"].append(self.best_fitness)
            self.history["best_solution"].append(self.best_solution)

            print(
                f"Generasi {iteration + 1}: Best Fitness = {self.best_fitness}, "
                f"Best Solution = {self.best_solution}"
            )

            # Pengecekan kriteria penghentian
            if self.stop_criteria is not None:
                if abs(self.best_fitness - self.stop_criteria) <= 0.1:
                    print("Kriteria penghentian tercapai.")
                    break

        return {
            "best_solution": self.best_solution,
            "best_fitness": self.best_fitness,
            "history": self.history,
        }

In [38]:
#inisiasi kma
n = 5
p = 0.5
d = 0.1
max_iter = 10

kma = KomodoMlipirAlgorithm(n, p, d, fitness_function, search_space, max_iter)
hasil = kma.optimize()
  
print(" ")
print(kma.history)
print(" ")
print(hasil)



Betina melakukan parthenogenesis.


Generasi 1: Best Fitness = 0.006034129690341546, Best Solution = [0.7611397  0.78606431 0.12811363]


Betina kawin dengan jantan besar.


Generasi 2: Best Fitness = 0.006034129690341546, Best Solution = [0.7611397  0.78606431 0.12811363]


Betina kawin dengan jantan besar.


...

Generasi 7: Best Fitness = 0.006034129690341546, Best Solution = [0.76986613 0.78333365 0.11233784]
Betina melakukan parthenogenesis.


Generasi 8: Best Fitness = 0.006034129690341546, Best Solution = [0.76915741 0.79408202 0.13613134]


Betina kawin dengan jantan besar.


Generasi 9: Best Fitness = 0.006034129690341546, Best Solution = [0.75048506 0.77540966 0.11745899]
Betina melakukan parthenogenesis.


Generasi 10: Best Fitness = 0.006034129690341546, Best Solution = [0.75315368 0.80310181 0.16737448]
 
{'best_fitness': [np.float64(0.006034129690341546), np.float64(0.006034129690341546), np.float64(0.006034129690341546), np.float64(0.006034129690341546), np.float64(0.006034129690341546), np.float64(0.006034129690341546), np.float64(0.006034129690341546), np.float64(0.006034129690341546), np.float64(0.006034129690341546), np.float64(0.006034129690341546)], 'best_solution': [array([0.7611397 , 0.78606431, 0.12811363]), array([0.7611397 , 0.78606431, 0.12811363]), array([0.86898909, 0.75231631, 0.        ]), array([0.83200611, 0.76388894, 0.        ]), array([0.79188208, 0.77644447, 0.07253699]), array([0.84680891, 0.75925688, 0.        ]), array([0.76986613, 0.78333365, 0.11233784]), array([0.76915741, 0.79408202, 0.13613134]), array([0.75048506, 0.77540966, 0.11745899]), array([0.75315368, 0.80310181, 0.16737448])]}
 
{'best_solution': array([0.75315368, 0.80310181, 0.16737448]), 'bes

In [40]:
def decode(individu):
    regparam_dict = {
        1:0,
        2:0.01,
        3:0.1,
        4:0.5,
        5:1
    }
    
    elasticnetparam_dict = {
        1:0,
        2:0.01,
        3:0.1,
        4:0.5,
        5:1
    }
    
    solver_dict = {
        1:"auto",
        2:"normal",
        3:"l-bfgs"
    }
    
    lower_bond = np.array([1,1,1])
    upper_bond = np.array([5,5,3])
    delta = (upper_bond - lower_bond)
    term = individu * delta
    hasil = np.int64(np.floor(lower_bond + term))
    
    regparam = regparam_dict[hasil[0]]
    elasticnetparam = elasticnetparam_dict[hasil[1]]
    solver = solver_dict[hasil[2]]
    
    return regparam, elasticnetparam, solver

In [41]:
regparam, elasticnetparam, solver = decode(hasil['best_solution'])

In [42]:
regparam, elasticnetparam, solver

(0.5, 0.5, 'auto')

In [44]:
lr_optimal = LinearRegression(featuresCol="scaled_features", labelCol="fare_amount", regParam=regparam, elasticNetParam=elasticnetparam, solver=solver)
model_optimal = lr_optimal.fit(train_data)
predictions_train_optimal = model_optimal.transform(train_data)
predictions_test_optimal = model_optimal.transform(test_data)
    
evaluator_optimal = RegressionEvaluator(labelCol="fare_amount", predictionCol="prediction", metricName="rmse")
rmse_train_optimal = evaluator.evaluate(predictions_train_optimal)
rmse_test_optimal = evaluator.evaluate(predictions_test_optimal)

print(f"RMSE Train Optimal: {rmse_train_optimal}")
print(f"RMSE Test Optimal: {rmse_test_optimal}")


RMSE Train Optimal: 19.518138713661962
RMSE Test Optimal: 24.913431513236596


In [45]:
delta = abs(rmse_train_optimal - rmse_test_optimal)
persentase = (delta / rmse_train_optimal) * 100
print(f"Persentase: {persentase}%")

Persentase: 27.64245545502826%


# Laporan link : https://drive.google.com/drive/folders/1tnz2zeFco_r9akKXdJKSYiAaHT63Rzgp?usp=sharing